<a href="https://colab.research.google.com/github/RonanD10/Tennis-Match-Prediction/blob/main/3_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
import random

from google.colab import drive, files
drive.mount('/content/drive')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

Mounted at /content/drive


In [ ]:
ALL_MATCHES = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_matches.csv')
all_players = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_players.csv')

<ipython-input-2-4c62233f470e>:1: DtypeWarning: Columns (9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  ALL_MATCHES = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_matches.csv')


In [ ]:
ALL_MATCHES.columns

Index(['index', 'tourney_id', 'tourney_name', 'surface', 'draw_size',
       'tourney_level', 'tourney_date', 'match_num', 'winner_id',
       'winner_seed', 'winner_entry', 'winner_name', 'winner_hand',
       'winner_ht', 'winner_ioc', 'winner_age', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc',
       'loser_age', 'score', 'best_of', 'round', 'minutes', 'w_ace', 'w_df',
       'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved',
       'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon',
       'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced', 'winner_rank',
       'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'winner_hard_elo_before', 'loser_hard_elo_before',
       'winner_hard_elo_after', 'loser_hard_elo_after',
       'winner_clay_elo_before', 'loser_clay_elo_before',
       'winner_clay_elo_after', 'loser_clay_elo_after',
       'winner_grass_elo_before', 'loser_grass_elo_before',
 

In [ ]:
# Table of features for model
# Engineer feature by feature below

# Drop unknown surface rows
ALL_MATCHES = ALL_MATCHES[ALL_MATCHES['surface'] != 'unknown']

# Drop unknown ranking rows
ALL_MATCHES = ALL_MATCHES[ALL_MATCHES['winner_rank'].isnull() == False]
ALL_MATCHES = ALL_MATCHES[ALL_MATCHES['loser_rank'].isnull() == False]
ALL_MATCHES = ALL_MATCHES.reset_index(drop=True)

# features = ALL_MATCHES[['surface', 'winner_rank', 'loser_rank']]

# NOTES
   # Remove pre-1970 data due to Elo crudeness and poor(?) data?

In [ ]:
# Surface Elo, Elo_surface_overall_avg

features['winner_elo_surface'] = 0
features['loser_elo_surface'] = 0
features['winner_elo'] = ALL_MATCHES['winner_elo_before']
features['loser_elo'] = ALL_MATCHES['loser_elo_before']

for i in range(len(ALL_MATCHES)):
    surface = ALL_MATCHES.loc[i, 'surface']
    features.at[i, 'winner_elo_surface'] = ALL_MATCHES.loc[i, f'winner_{surface}_elo_before']
    features.at[i, 'loser_elo_surface'] = ALL_MATCHES.loc[i, f'loser_{surface}_elo_before']

features['winner_avged_elo'] = (features['winner_elo_surface'] + features['winner_elo']) / 2
features['loser_avged_elo'] = (features['loser_elo_surface'] + features['loser_elo']) / 2

<ipython-input-5-309a8245bf48>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['winner_elo_surface'] = 0
<ipython-input-5-309a8245bf48>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['loser_elo_surface'] = 0
<ipython-input-5-309a8245bf48>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [ ]:
# h2h functions

def h2h(p1_id, p2_id, i):
    """
    Returns p1 wins, p2 wins prior to match
    """
    df = ALL_MATCHES.iloc[:i]
    p1_wins = len(df[(df['winner_id'] == p1_id) & (df['loser_id'] == p2_id)])
    p2_wins = len(df[(df['winner_id'] == p2_id) & (df['loser_id'] == p1_id)])

    return p1_wins, p2_wins


def surface_h2h(p1_id, p2_id, surface, i):
    """
    Returns surface p1 wins, p2 wins prior to match
    """
    df = ALL_MATCHES.iloc[:i]
    p1_wins = len(df[(df['winner_id'] == p1_id) & (df['loser_id'] == p2_id) & (df['surface'] == surface)])
    p2_wins = len(df[(df['winner_id'] == p2_id) & (df['loser_id'] == p1_id) & (df['surface'] == surface)])

    return p1_wins, p2_wins

In [ ]:
# Add h2h

features['winner_wins'] = 0
features['loser_wins'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(i/len(ALL_MATCHES) * 100, ' %')
    winner_id = ALL_MATCHES.loc[i, 'winner_id']
    loser_id = ALL_MATCHES.loc[i, 'loser_id']
    winner_wins, loser_wins = h2h(winner_id, loser_id, i)
    features.at[i, 'winner_wins'] = winner_wins
    features.at[i, 'loser_wins'] = loser_wins

99.55821044116732  %


In [ ]:
# Add surface h2h

features['winner_wins_surface'] = 0
features['loser_wins_surface'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/len(ALL_MATCHES) * 100, 1), ' %')
    winner_id = ALL_MATCHES.loc[i, 'winner_id']
    loser_id = ALL_MATCHES.loc[i, 'loser_id']
    surface = ALL_MATCHES.loc[i, 'surface']
    winner_wins, loser_wins = surface_h2h(winner_id, loser_id, surface, i)
    features.at[i, 'winner_wins_surface'] = winner_wins
    features.at[i, 'loser_wins_surface'] = loser_wins

99.6  %


In [ ]:
# Surface form, season form

def surface_form(player_id, surface, date):
    """
    Returns win % over last 12 months on a given surface.
    Date format: YYYYMMDD.
    """
    df = ALL_MATCHES
    year_ago = date - 10000
    num_wins = len(df[(df['winner_id'] == player_id) & (df['surface'] == surface) & (df['tourney_date'] > year_ago) & (df['tourney_date'] <= date)])
    num_losses = len(df[(df['loser_id'] == player_id) & (df['surface'] == surface) & (df['tourney_date'] > year_ago) & (df['tourney_date'] <= date)])

    return num_wins / (num_wins + num_losses)


def season_form(player_id, date):
    """
    Returns win % over last 12 months.
    Date format: YYYYMMDD.
    """
    df = ALL_MATCHES
    year_ago = date - 10000
    num_wins = len(df[(df['winner_id'] == player_id) & (df['tourney_date'] > year_ago) & (df['tourney_date'] <= date)])
    num_losses = len(df[(df['loser_id'] == player_id) & (df['tourney_date'] > year_ago) & (df['tourney_date'] <= date)])

    return num_wins / (num_wins + num_losses)

In [ ]:
# Add season and surface form features

features['winner_12month_form'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/len(ALL_MATCHES) * 100, 1), ' %')

    winner_id = ALL_MATCHES.loc[i, 'winner_id']
    tourney_date = ALL_MATCHES.loc[i, 'tourney_date']

    winner_form = season_form(winner_id, tourney_date)
    features.at[i, 'winner_12month_form'] = winner_form

99.6  %


In [ ]:
features['loser_12month_form'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/len(ALL_MATCHES) * 100, 1), ' %')

    loser_id = ALL_MATCHES.loc[i, 'loser_id']
    tourney_date = ALL_MATCHES.loc[i, 'tourney_date']

    loser_form = season_form(loser_id, tourney_date)
    features.at[i, 'loser_12month_form'] = loser_form

99.6  %


In [ ]:
features['winner_12month_form_surface'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/len(ALL_MATCHES) * 100, 1), ' %')

    winner_id = ALL_MATCHES.loc[i, 'winner_id']
    tourney_date = ALL_MATCHES.loc[i, 'tourney_date']
    surface = ALL_MATCHES.loc[i, 'surface']

    winner_form_surface = surface_form(winner_id, surface, tourney_date)
    features.at[i, 'winner_12month_form_surface'] = winner_form_surface

99.6  %


In [ ]:
features['loser_12month_form_surface'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/len(ALL_MATCHES) * 100, 1), ' %')

    loser_id = ALL_MATCHES.loc[i, 'loser_id']
    tourney_date = ALL_MATCHES.loc[i, 'tourney_date']
    surface = ALL_MATCHES.loc[i, 'surface']

    loser_form_surface = surface_form(loser_id, surface, tourney_date)
    features.at[i, 'loser_12month_form_surface'] = loser_form_surface

99.6  %


In [ ]:
features['winner_hand'] = ALL_MATCHES['winner_hand'].map({'R': 0, 'L': 1})
features['loser_hand'] = ALL_MATCHES['loser_hand'].map({'R': 0, 'L': 1})
features

,surface,winner_rank,loser_rank,winner_elo_surface,loser_elo_surface,winner_elo,loser_elo,winner_avged_elo,loser_avged_elo,winner_wins,loser_wins,winner_wins_surface,loser_wins_surface,winner_12month_form,loser_12month_form,winner_12month_form_surface,loser_12month_form_surface,winner_hand,loser_hand
0,grass,3.0,23.0,1988.523,1644.770,2151.654,1772.521,2070.0885,1708.6455,0,0,0,0,0.833333,0.000000,0.833333,0.000000,0.0,0.0
1,grass,22.0,30.0,1584.360,1501.014,1859.956,1714.681,1722.1580,1607.8475,0,0,0,0,0.666667,0.000000,0.666667,0.000000,1.0,0.0
2,grass,55.0,58.0,1554.535,1792.652,1712.486,1821.925,1633.5105,1807.2885,0,0,0,0,0.500000,0.000000,0.500000,0.000000,0.0,0.0
3,grass,15.0,39.0,1740.677,1542.475,1885.650,1794.640,1813.1635,1668.5575,0,0,0,0,0.750000,0.000000,0.750000,0.000000,0.0,0.0
4,grass,18.0,97.0,1766.935,1503.159,1922.537,1607.397,1844.7360,1555.2780,0,0,0,0,0.666667,0.000000,0.666667,0.000000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144634,clay,7.0,56.0,1901.149,1590.551,1926.081,1684.596,1913.6150,1637.5735,0,0,0,0,0.717949,0.515152,0.783784,0.533333,0.0,0.0
144635,clay,29.0,47.0,1694.263,1672.773,1724.848,1698.955,1709.5555,1685.8640,0,0,0,0,0.525424,0.680000,0.617647,0.708333,0.0,0.0
144636,clay,117.0,19.0,1542.649,1554.270,1567.658,1797.649,1555.1535,1675.9595,0,1,0,0,0.700000,0.603448,0.833333,0.500000,0.0,0.0
144637,clay,7.0,44.0,1905.735,1605.691,1932.462,1796.400,1919.0985,1701.0455,1,0,0,0,0.717949,0.636364,0.783784,0.666667,0.0,0.0


In [ ]:
names_dict = {'winner_rank': 'A_rank', 'loser_rank': 'B_rank',
              'winner_elo_surface': 'A_elo_surface', 'loser_elo_surface': 'B_elo_surface',
              'winner_elo': 'A_elo', 'loser_elo': 'B_elo',
              'winner_avged_elo': 'A_avged_elo', 'loser_avged_elo': 'B_avged_elo',
              'winner_wins': 'A_wins', 'loser_wins': 'B_wins',
              'winner_wins_surface': 'A_wins_surface', 'loser_wins_surface': 'B_wins_surface',
              'winner_12month_form': 'A_12month_form', 'loser_12month_form': 'B_12month_form',
              'winner_12month_form_surface': 'A_12month_form_surface', 'loser_12month_form_surface': 'B_12month_form_surface',
              'surface': 'surface', 'winner_hand': 'A_hand', 'loser_hand': 'B_hand'}

features = features.rename(columns=names_dict)

NameError: name 'features' is not defined

In [ ]:
# Add tourney level and round

features['winner_hand'] = ALL_MATCHES['winner_hand']
features['loser_hand'] = ALL_MATCHES['loser_hand']
features['tourney_level'] = ALL_MATCHES['tourney_level']
features['round'] = ALL_MATCHES['round']
features.dropna(inplace=True)
features.reset_index(drop=True, inplace=True)

In [ ]:
# Randomly assign winner and loser to player A and B, whilst maintaining attributes

features['A_won'] = 0
np.random.seed(seed=42)

for i in range(len(features)):
    playerA_won = np.random.randint(0, 2)
    features.loc[i, 'A_won'] = playerA_won
    if playerA_won == 0:
        features.loc[i, 'A_rank'], features.loc[i, 'B_rank'] = features.loc[i, 'B_rank'], features.loc[i, 'A_rank']
        features.loc[i, 'A_elo'], features.loc[i, 'B_elo'] = features.loc[i, 'B_elo'], features.loc[i, 'A_elo']
        features.loc[i, 'A_elo_surface'], features.loc[i, 'B_elo_surface'] = features.loc[i, 'B_elo_surface'], features.loc[i, 'A_elo_surface']
        features.loc[i, 'A_wins'], features.loc[i, 'B_wins'] = features.loc[i, 'B_wins'], features.loc[i, 'A_wins']
        features.loc[i, 'A_wins_surface'], features.loc[i, 'B_wins_surface'] = features.loc[i, 'B_wins_surface'], features.loc[i, 'A_wins_surface']
        features.loc[i, 'A_12month_form'], features.loc[i, 'B_12month_form'] = features.loc[i, 'B_12month_form'], features.loc[i, 'A_12month_form']
        features.loc[i, 'A_12month_form_surface'], features.loc[i, 'B_12month_form_surface'] =  features.loc[i, 'B_12month_form_surface'], features.loc[i, 'A_12month_form_surface']
        features.loc[i, 'A_hand'], features.loc[i, 'B_hand'] = features.loc[i, 'B_hand'], features.loc[i, 'A_hand']

In [ ]:
# Assign grand slam rounds
slams = ALL_MATCHES[ALL_MATCHES['tourney_level'] == 'G']
slams = slams[(slams['winner_rank'].isna() == False) & (slams['loser_rank'].isna() == False)]
slams.reset_index(drop=True, inplace=True)

f = features
f['slam_round'] = 0

for i in range(len(slams)):
    w_elo = slams.loc[i, 'winner_elo_before']
    l_elo = slams.loc[i, 'loser_elo_before']
    w_rank = slams.loc[i, 'winner_rank']
    l_rank = slams.loc[i, 'loser_rank']
    round = slams.loc[i, 'round']
    index = f[(f['A_elo'] == w_elo) & (f['B_elo'] == l_elo) & (f['A_rank'] == w_rank) & (f['B_rank'] == l_rank)].index[0]
    f.loc[index, 'slam_round'] = round

In [ ]:
features['elo_difference'] = abs(features['A_elo'] - features['B_elo'])
features = pd.get_dummies(features, columns=['surface'])
features = features.dropna().reset_index(drop=True)

features[['surface_hard', 'surface_carpet', 'surface_grass', 'surface_clay']] = features[['surface_hard', 'surface_carpet', 'surface_grass', 'surface_clay']].astype(int)

In [ ]:
features.to_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/features_processed_random.csv', index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Informed from model results...

# Add 6-month, 3-month, 1-month form + surface form to emphasise form imporance

ALL_MATCHES = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_matches.csv')
features_new = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/features_processed_random_new.csv')

# Drop unknown surface rows
ALL_MATCHES = ALL_MATCHES[ALL_MATCHES['surface'] != 'unknown']

from pandas.tseries.offsets import DateOffset
ALL_MATCHES['tourney_date'] = pd.to_datetime(ALL_MATCHES['tourney_date'], format='%Y%m%d')
ALL_MATCHES['date_minus_1'] = ALL_MATCHES.tourney_date - DateOffset(months=1)
ALL_MATCHES['date_minus_3'] = ALL_MATCHES.tourney_date - DateOffset(months=3)
ALL_MATCHES['date_minus_6'] = ALL_MATCHES.tourney_date - DateOffset(months=6)

def surface_form_months(i, player_id, surface, date, months):
    """
    i is row index.
    Returns win % of player_id over last (months)months on a given surface.
    Date format: 'YYYY-MM-DD'.
    """
    df = ALL_MATCHES
    past_date = ALL_MATCHES.loc[i, f'date_minus_{months}']
    num_wins = len(df[(df['winner_id'] == player_id) & (df['surface'] == surface) & (df['tourney_date'] > past_date ) & (df['tourney_date'] <= date)])
    num_losses = len(df[(df['loser_id'] == player_id) & (df['surface'] == surface) & (df['tourney_date'] > past_date) & (df['tourney_date'] <= date)])

    return num_wins / (num_wins + num_losses)


def season_form_months(i, player_id, date, months):
    """
    i is row index.
    Returns win % of player_id over last (months)months.
    Date format: 'YYYY-MM-DD'.
    """
    df = ALL_MATCHES
    past_date = ALL_MATCHES.loc[i, f'date_minus_{months}']
    num_wins = len(df[(df['winner_id'] == player_id) & (df['tourney_date'] > past_date ) & (df['tourney_date'] <= date)])
    num_losses = len(df[(df['loser_id'] == player_id) & (df['tourney_date'] > past_date) & (df['tourney_date'] <= date)])

    return num_wins / (num_wins + num_losses)

<ipython-input-60-d3c81a143c60>:5: DtypeWarning: Columns (9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  ALL_MATCHES = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_matches.csv')


In [ ]:
ALL_MATCHES[ALL_MATCHES['tourney_date'] > '2000-01-01']

,index,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,winner_hard_elo_before,loser_hard_elo_before,winner_hard_elo_after,loser_hard_elo_after,winner_clay_elo_before,loser_clay_elo_before,winner_clay_elo_after,loser_clay_elo_after,winner_grass_elo_before,loser_grass_elo_before,winner_grass_elo_after,loser_grass_elo_after,winner_carpet_elo_before,loser_carpet_elo_before,winner_carpet_elo_after,loser_carpet_elo_after,winner_elo_before,loser_elo_before,winner_elo_after,loser_elo_after,date_minus_1,date_minus_3,date_minus_6,winner_6month_form_surface,loser_6month_form_surface,winner_12month_form,loser_12month_form,winner_6month_form,loser_6month_form
120090,591,2000-339,Adelaide,hard,32.0,A,2000-01-03,1,102358,1.0,NaN,Thomas Enqvist,R,190.0,SWE,25.7,103096,NaN,NaN,Arnaud Clement,R,173.0,FRA,22.0,6-3 6-4,3,3,76.0,6.0,0.0,66.0,29.0,23.0,23.0,10.0,2.0,2.0,1.0,3.0,59.0,37.0,25.0,13.0,9.0,2.0,4.0,4.0,2606.0,56.0,805.0,1897.846,1669.502,1904.622,1662.726,1685.369,1524.054,1500.000,1500.000,1537.490,1508.863,1500.0,1500.0,1697.820,1500.368,1500.0,1500.0,1912.685,1682.009,1919.389,1675.305,1999-12-03,1999-10-03,1999-07-03,0.0,0,NaN,NaN,NaN,NaN
120091,592,2000-339,Adelaide,hard,32.0,A,2000-01-03,2,103819,NaN,NaN,Roger Federer,R,185.0,SUI,18.3,102533,NaN,NaN,Jens Knippschild,R,190.0,GER,24.8,6-1 6-4,3,3,45.0,6.0,3.0,46.0,28.0,24.0,12.0,9.0,0.0,0.0,3.0,5.0,42.0,15.0,13.0,12.0,8.0,0.0,3.0,64.0,749.0,91.0,525.0,1615.411,1578.959,1629.738,1564.632,1449.244,1596.466,1500.000,1500.000,1475.687,1528.757,1500.0,1500.0,1553.975,1488.708,1500.0,1500.0,1665.130,1667.865,1681.256,1651.739,1999-12-03,1999-10-03,1999-07-03,0.0,0,NaN,NaN,NaN,NaN
120092,593,2000-339,Adelaide,hard,32.0,A,2000-01-03,3,102998,NaN,NaN,Jan Michael Gambill,R,190.0,USA,22.5,101885,NaN,NaN,Wayne Arthurs,L,190.0,AUS,28.7,3-6 7-6(5) 6-4,3,3,115.0,8.0,3.0,81.0,40.0,35.0,28.0,15.0,0.0,1.0,26.0,2.0,103.0,59.0,49.0,22.0,16.0,4.0,5.0,58.0,803.0,105.0,449.0,1761.124,1520.736,1767.537,1514.323,1473.661,1534.106,1500.000,1500.000,1508.123,1613.178,1500.0,1500.0,1506.482,1500.000,1500.0,1500.0,1774.217,1679.014,1785.939,1667.292,1999-12-03,1999-10-03,1999-07-03,0.0,0,NaN,NaN,NaN,NaN
120093,594,2000-339,Adelaide,hard,32.0,A,2000-01-03,4,103206,7.0,NaN,Sebastien Grosjean,R,175.0,FRA,21.5,102776,NaN,NaN,Andrew Ilie,R,180.0,AUS,23.6,6-2 6-1,3,3,65.0,4.0,2.0,66.0,35.0,28.0,14.0,7.0,4.0,4.0,0.0,3.0,49.0,22.0,12.0,8.0,8.0,1.0,6.0,27.0,1298.0,54.0,845.0,1692.710,1611.315,1705.029,1598.996,1625.805,1582.347,1500.000,1500.000,1586.556,1524.785,1500.0,1500.0,1507.317,1468.340,1500.0,1500.0,1768.528,1647.110,1779.154,1636.484,1999-12-03,1999-10-03,1999-07-03,0.0,0,NaN,NaN,NaN,NaN
120094,595,2000-339,Adelaide,hard,32.0,A,2000-01-03,5,102796,3.0,NaN,Magnus Norman,R,188.0,SWE,23.5,102401,NaN,WC,Scott Draper,L,178.0,AUS,25.5,6-4 6-4,3,3,68.0,6.0,2.0,52.0,32.0,26.0,12.0,10.0,0.0,1.0,4.0,2.0,73.0,40.0,25.0,16.0,10.0,7.0,10.0,15.0,1748.0,154.0,297.0,1767.878,1653.496,1778.793,1642.581,1795.024,1534.834,1500.000,1500.000,1514.435,1596.359,1500.0,1500.0,1592.328,1512.397,1500.0,1500.0,1887.794,1659.638,1894.576,1652.856,1999-12-03,1999-10-03,1999-07-03,0.0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193332,1250,2024-0322,Geneva,clay,32.0,A,2024-05-20,298,134770,2,NaN,Casper Ruud,R,183.0,NOR,25.4

In [ ]:
# Add 6-month surface form

ALL_MATCHES['winner_6month_form_surface'] = 0

for i in range(120090, len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/ 70000, 1), ' %')

    winner_id = ALL_MATCHES.loc[i, 'winner_id']
    date = ALL_MATCHES.loc[i, 'tourney_date']
    surface = ALL_MATCHES.loc[i, 'surface']

    winner_form = surface_form_months(i, winner_id, surface, date, 6)

    ALL_MATCHES.at[i, 'winner_6month_form_surface'] = winner_form

In [ ]:
ALL_MATCHES['loser_6month_form_surface'] = 0

for i in range(120090, len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/ 70000, 1), ' %')

    loser_id = ALL_MATCHES.loc[i, 'loser_id']
    date = ALL_MATCHES.loc[i, 'tourney_date']
    surface = ALL_MATCHES.loc[i, 'surface']

    loser_form = surface_form_months(i, loser_id, surface, date, 6)

    ALL_MATCHES.at[i, 'loser_6month_form_surface'] = loser_form

In [ ]:
# Add 3-month surface form

ALL_MATCHES['winner_3month_form_surface'] = 0
ALL_MATCHES['loser_3month_form_surface'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/len(ALL_MATCHES) * 100, 1), ' %')

    winner_id = ALL_MATCHES.loc[i, 'winner_id']
    loser_id = ALL_MATCHES.loc[i, 'loser_id']
    date = ALL_MATCHES.loc[i, 'tourney_date']
    surface = ALL_MATCHES.loc[i, 'surface']

    winner_form = surface_form_months(i, winner_id, surface, date, 3)
    loser_form = surface_form_months(i, loser_id, surface, date, 3)

    ALL_MATCHES.at[i, 'winner_3month_form'] = winner_form
    ALL_MATCHES.at[i, 'loser_3month_form'] = loser_form

In [ ]:
# Add 1-month surface form

ALL_MATCHES['winner_6month_form_surface'] = 0
ALL_MATCHES['loser_6month_form_surface'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/len(ALL_MATCHES) * 100, 1), ' %')

    winner_id = ALL_MATCHES.loc[i, 'winner_id']
    loser_id = ALL_MATCHES.loc[i, 'loser_id']
    date = ALL_MATCHES.loc[i, 'tourney_date']
    surface = ALL_MATCHES.loc[i, 'surface']

    winner_form = surface_form_months(i, winner_id, surface, date, 1)
    loser_form = surface_form_months(i, loser_id, surface, date, 1)

    ALL_MATCHES.at[i, 'winner_1month_form'] = winner_form
    ALL_MATCHES.at[i, 'loser_1month_form'] = loser_form

In [ ]:
### Possible additions:
# Recovering from injury -- T/F
# Performances in same tournament level
# Highest previous round in tournament
# Seeding (if available)

def injury(player_id, date):
    """
    Returns True if <=10 matches in last 12 months; False otherwise.
    """
    df = ALL_MATCHES
    year_ago = date - 10000
    num_matches = len(df[(df['winner_id'] == player_id) & (df['tourney_date'] > year_ago)] & (df['tourney_date'] < date))

    return num_matches <= 10
